In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import pandas as pd

# Define Models
def langmuir(C, K, Qmax):
    return (Qmax * K * C) / (1 + K * C)

def hill(C, Kd, n, Qmax):
    return Qmax * (C * n) / (Kd * n + C ** n)

def davis(C, K, n, Qmax):
    return (Qmax * (K * C) * n) / (1 + (K * C) * n)

# Example Data: replace with your data
adsorption_data_by_temp = {
    300: {'C': np.array([x1, x2, x3, x4, x5, x6]), 'Q': np.array([y1, y2, y3, y4, y5, y6])},
    325: {'C': np.array([x1, x2, x3, x4, x5, x6]), 'Q': np.array([y1, y2, y3, y4, y5, y6])},
    350: {'C': np.array([x1, x2, x3, x4, x5, x6]), 'Q': np.array([y1, y2, y3, y4, y5, y6])},
    375: {'C': np.array([x1, x2, x3, x4, x5, x6]), 'Q': np.array([y1, y2, y3, y4, y5, y6])}
}

# Fit Models & Plot
fit_results = []
colors = ['red', 'blue', 'green', 'purple']
plt.figure(figsize=(10, 6))

for idx, (T, data) in enumerate(adsorption_data_by_temp.items()):
    C, Q = data['C'], data['Q']
    popt_lang, _ = curve_fit(langmuir, C, Q, bounds=(0, np.inf))
    popt_hill, _ = curve_fit(hill, C, Q, bounds=(0, np.inf))
    popt_davis, _ = curve_fit(davis, C, Q, bounds=(0, np.inf))
    R2_lang = r2_score(Q, langmuir(C, *popt_lang))
    R2_hill = r2_score(Q, hill(C, *popt_hill))
    R2_davis = r2_score(Q, davis(C, *popt_davis))
    fit_results.append({'Temperature (K)': T, 'Langmuir R²': R2_lang, 'Hill R²': R2_hill, 'Davis R²': R2_davis})
    C_fit = np.linspace(0.001, 0.45, 300)
    plt.plot(C_fit, langmuir(C_fit, *popt_lang), '--', color=colors[idx], label=f'Langmuir {T}K (R²={R2_lang:.4f})')
    plt.plot(C_fit, hill(C_fit, *popt_hill), '-', color=colors[idx], label=f'Hill {T}K (R²={R2_hill:.4f})')
    plt.plot(C_fit, davis(C_fit, *popt_davis), ':', color=colors[idx], label=f'Davis {T}K (R²={R2_davis:.4f})')
    plt.scatter(C, Q, color=colors[idx], edgecolor='black', label=f'Exp Data {T}K', zorder=5)

plt.xlabel('Urea Concentration (M)')
plt.ylabel('Adsorption (Q)')

plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('urea_binding_model_fits1.png', dpi=300)
plt.show()

df_results = pd.DataFrame(fit_results)
df_results.to_csv('best_model_fit_results.csv', index=False)
print("Saved plot to 'urea_binding_model_fits1.png'")
print("Saved fit data to 'best_model_fit_results.csv'")
print(df_results.to_string(index=False))